# Running the gerrychain to find a map without holes on congressional districts

@authors: vcle, bpuhani

In [ ]:
import io
import random
from contextlib import redirect_stdout

import maup
import pandas as pd
from shapely.ops import unary_union

import utilities as util
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
from gerrychain import Graph, Partition, proposals, updaters, constraints, accept, MarkovChain, Election
from gerrychain.tree import bipartition_tree
from gerrychain.updaters import cut_edges, Tally
from gerrychain.proposals import recom, propose_random_flip
from gerrychain.accept import always_accept
from functools import partial
from gerrychain.metrics import efficiency_gap  # get the efficiency gap directly from gerrychain

## Loading the needed data.
For this notebook to work we assume, that you ran the following notebooks first:
* `0_IL_import_and_explore_data.ipynb`
* `B_2_IL_clean_maup_with_congress.ipynb`

In [ ]:
il_df: gpd.GeoDataFrame = util.load_shapefile("il_data/IL_congress_without_holes.shp")
il_graph: Graph = util.load_graph("il_data/IL_congress_without_holes.shp")

In [ ]:
partition_at_5_000: Partition
partition_at_10_000: Partition
partition_at_15_000: Partition
partition_at_20_000: Partition
partition_at_25_000: Partition
partition_at_30_000: Partition
partition_at_35_000: Partition
partition_at_40_000: Partition

Setup Updaters

In [ ]:
def has_holes(partition, district) -> bool:
    # Merge all geometries in the district into a single polygon/multipolygon
    raw_geometry = unary_union([partition.graph.nodes[v]["geometry"]
                                for v in partition.parts[district]])

    # Try to repair invalid geometry
    geom_fixed = raw_geometry.buffer(0)

    # A simple hole check: does the geometry have interior rings?
    # (for Polygon: check .interiors; for MultiPolygon: check if any part has interiors)
    if geom_fixed.geom_type == "Polygon":
        return len(geom_fixed.interiors) > 0
    elif geom_fixed.geom_type == "MultiPolygon":
        return any(len(p.interiors) > 0 for p in geom_fixed.geoms)
    else:
        print(f"Not a polygon geometry: {geom_fixed.geom_type}")
        return False  # Not a polygon geometry? Then we ignore it.

In [ ]:
il_updaters = {
    "total_population": Tally("TOTPOP", alias="total_population"),
    # "hisp_population": Tally("HISP", alias="hisp_population"), # not needed apparently
    "cut_edges": cut_edges,
    # calculate if a district has holes
    "district_has_holes": lambda p: [int(has_holes(p, d)) for d in p.parts],
}

In [ ]:
elections = [
    Election("PRE20", {"Dem": "G20PRED", "Rep": "G20PRER"}),
    Election("USS20", {"Dem": "G20USSD", "Rep": "G20USSR"}),
]

In [ ]:
# adding the elections to the updaters
election_updaters = {election.name: election for election in elections}
il_updaters.update(election_updaters)

In [ ]:
# Set up the initial partition object
initial_partition = Partition(
    il_graph,
    assignment="district",  # use the "district" column because this is the new one without holes.
    updaters=il_updaters,
)

In [ ]:
# Define the ideal population
ideal_population = sum(initial_partition["total_population"].values()) / len(initial_partition)
print("Nr of districts:", len(initial_partition))
print("Ideal population:", ideal_population)

In [ ]:
# Define the recom proposal
proposal = partial(
    recom,
    pop_col="TOTPOP",
    pop_target=ideal_population,
    epsilon=0.02,
    method=partial(
        bipartition_tree,
        max_attempts=100,
        allow_pair_reselection=True
    )
)

In [ ]:
# define the lists that are needed to track the changes
list_of_nr_of_cut_edges = []

list_of_dem_won_districts_pre20 = []
list_of_dem_won_districts_uss20 = []

list_of_eg_pre20 = []
list_of_eg_uss20 = []

list_of_dem_percents_pre20 = []
list_of_dem_percents_uss20 = []

In [ ]:
# create a checkpoint for all the lists in one big dictionary
checkpoint_dict = {
    "list_of_nr_of_cut_edges": list_of_nr_of_cut_edges,
    "list_of_dem_won_districts_pre20": list_of_dem_won_districts_pre20,
    "list_of_dem_won_districts_uss20": list_of_dem_won_districts_uss20,
    "list_of_eg_pre20": list_of_eg_pre20,
    "list_of_eg_uss20": list_of_eg_uss20,
    "list_of_dem_percents_pre20": list_of_dem_percents_pre20,
    "list_of_dem_percents_uss20": list_of_dem_percents_uss20
}

In [ ]:
def run_the_chain(nr_of_total_steps: int, start_partition: Partition, offset: int = 0) -> Partition:
    """Runs the chain for the specified number of steps. Returns the last partition"""

    # Set up the chain
    chain = MarkovChain(
        proposal=proposal,
        constraints=[
            # Compactness constraint
            constraints.UpperBound(lambda p: len(p["cut_edges"]), 2 * len(initial_partition["cut_edges"])),
            # Population constraint
            constraints.within_percent_of_ideal_population(initial_partition, 0.02, "total_population"),
            # set constraint for the map not to allow holes (lower and upper bound is 1 == (True) == no Holes)
            constraints.Bounds(lambda p: p["district_has_holes"], (0, 0))
        ],
        accept=always_accept,
        initial_state=start_partition,
        total_steps=nr_of_total_steps - offset
    )
    last_partition: Partition = start_partition

    for (i, partition) in enumerate(chain.with_progress_bar()):
        last_partition = partition

        # Calculate and append the efficiency gap values for each election to checkpoint_dict
        checkpoint_dict["list_of_eg_pre20"].append(efficiency_gap(partition["PRE20"]))
        checkpoint_dict["list_of_eg_uss20"].append(efficiency_gap(partition["USS20"]))

        # Append the sorted percentages of Democratic votes for each election to checkpoint_dict
        checkpoint_dict["list_of_dem_percents_pre20"].append(sorted(partition["PRE20"].percents("Dem")))
        checkpoint_dict["list_of_dem_percents_uss20"].append(sorted(partition["USS20"].percents("Dem")))

        # Append the number of districts won by the Democratic Party for each election to checkpoint_dict
        checkpoint_dict["list_of_dem_won_districts_pre20"].append(partition["PRE20"].wins("Dem"))
        checkpoint_dict["list_of_dem_won_districts_uss20"].append(partition["USS20"].wins("Dem"))

        # Append the number of cut edges for this partition to checkpoint_dict
        checkpoint_dict["list_of_nr_of_cut_edges"].append(len(partition["cut_edges"]))

    return last_partition

## RUN FIRST 5_000 STEPS

In [ ]:
partition_at_5_000 = run_the_chain(5_000, initial_partition)

### Saving the progress for the first 5_000 steps

In [ ]:
# load the checkpoint if it exists
checkpoint_dict = util.checkpoint("IL_plot_results_5_000", checkpoint_dict)

In [ ]:
assignment_at_5_000 = util.checkpoint("IL_Gerrychain_step_5_000", partition_at_5_000)
partition_at_5_000 = Partition(
    graph=il_graph,
    assignment=assignment_at_5_000,
    updaters=il_updaters,
)

In [ ]:
print(assignment_at_5_000)

## RUN NEXT 5_000 STEPS

In [ ]:
partition_at_10_000 = run_the_chain(10_000, partition_at_5_000, 5_000)

### save the progress for the next 5_000 steps

In [ ]:
assignment_at_10_000 = util.checkpoint("IL_Gerrychain_step_10_000", partition_at_10_000)
partition_at_10_000 = Partition(
    graph=il_graph,
    assignment=assignment_at_10_000,
    updaters=il_updaters,
)

In [ ]:
# load the checkpoint if it exists
checkpoint_dict = util.checkpoint("IL_plot_results_10_000", checkpoint_dict)

## Run the next 5_000 steps

In [ ]:
partition_at_15_000 = run_the_chain(15_000, partition_at_10_000, 10_000)

### save the progress for the next 5_000 steps

In [ ]:
assignment_at_15_000 = util.checkpoint("IL_Gerrychain_step_15_000", partition_at_15_000)
partition_at_15_000 = Partition(
    graph=il_graph,
    assignment=assignment_at_15_000,
    updaters=il_updaters,
)